In [ ]:
import numpy as np
from numpy import sin, pi
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats
import ultranest
#Import packages used
import math
import matplotlib.pylab as pl
from scipy.integrate import solve_ivp
from sklearn.preprocessing import normalize
import pandas as pd

In [ ]:
#Define the ODEs
def Fun_Logistic(t, y, mu_max, rho, R_0):    
    dN = mu_max*y[0]*(1 - y[0]/(R_0*rho))
    
    return [dN]

In [ ]:
file_name = '/Users/isalinelucille-guex/Documents/CoCulture_Soil/Data/Phil_S20_community_growth.xlsx' # path to file + file name
sheet =  5 # sheet name or sheet number or list of sheet numbers and names

#Time settings for the solver
tSteps_1 = np.multiply([0,0,0,0,1,1,1,1,3,3,3,3,7,7,7,7,15,15,15,15,22,22,22,22],24)
tSteps = np.multiply([0,1,3,7,15,22],24)
tSpan = [0, 528]

R_0 = 1.5*1e-04

df = pd.read_excel(io=file_name, sheet_name=sheet)
#print(df.head())  # print first 5 rows of the dataframe

#print(df.to_numpy())

Data = df.to_numpy()

Data = Data[0,1:25]

yerr = 1e-10

#Initial conditions (initial abundances of the two variables: [microbes,resource])
initConds = [Data[0]]

print(tSteps)

In [ ]:
# Plot the result

#Set size of plot
plt.rcParams["figure.figsize"] = (8,3)

#Get handles to the figure and the current set of axes
fig_CRM1, ax = plt.subplots()

#Set x and y limits of plot
ax.set_xlim(0,550)
ax.set_ylim(0,4)

#Set color palette
plotColors = pl.cm.tab20b(np.linspace(0,1,10))

#Format the plot
plt.rcParams["figure.figsize"] = (8,3)
fig_CRM5, ax = plt.subplots()
ax.set_xlim(tSpan[0],tSpan[-1])

#Plot population dynamics
ax.plot(tSteps,Data, color=plotColors[0], label='Microbial population') #First rows of solCRM.y contain population timecourses

#Create legend and label axes
plt.legend(loc='upper right')
plt.xlabel("Time")
plt.ylabel("Concentration")

In [ ]:
parameters1 = ['mu_max', 'rho']

def prior_transform1(cube):
    # the argument, cube, consists of values from 0 to 1
    # we have to convert them to physical scales

    params = cube.copy()
    #print(cube)
    # let background level go from 0 to +5
    params[0] = cube[0]
    # let background level go from 0 to +1
    params[1] = cube[1]

    return params

In [ ]:
def log_likelihood1(params):
    mu_max, rho = params
    inArgs_model = [mu_max, rho, R_0]
    
    #tSteps_temp = np.multiply([0,1,3,7,15,22],24)

    # compute for each x point, where it should lie in y
    solCRM_model = solve_ivp(fun=Fun_Logistic, t_span=tSpan, t_eval=tSteps, y0=initConds, args=inArgs_model)
    y_model = np.squeeze(np.asarray(solCRM_model.y[0:6]))
    #y_model = np.repeat(y_model, 4)
    print(y_model)
    loglike = -0.5 * (((y_model - Data))**2).sum()


    return loglike

In [ ]:
sampler1 = ultranest.ReactiveNestedSampler(
    parameters1,
    log_likelihood1, 
    prior_transform1
    #wrapped_params=[False, False],
)

result1 = sampler1.run(min_num_live_points=400)
sampler1.print_results()

In [ ]:
from ultranest.plot import cornerplot
cornerplot(result1)
sampler1.ncall

plt.figure()
plt.title("1-sine fit")
plt.xlabel('x')
plt.ylabel('y')
plt.errorbar(x=tSteps, y=test_data[0,:], yerr=yerr,
             marker='o', ls=' ', color='orange')

t_grid = np.linspace(0, 10, 550)

from ultranest.plot import PredictionBand
band = PredictionBand(t_grid)


# go through the solutions
for mu_max, rho in sampler1.results['samples']:
    # compute for each time the y value
    inArgs_fin = [mu_max, rho, R_0]
    solCRM_model_fin = solve_ivp(fun=Fun_Logistic, t_span=tSpan, t_eval=t_grid, y0=initConds, args=inArgs_fin)
    y_model = solCRM_model_fin.y[0,:]
    
    band.add(y_model)

band.line(color='k')
# add 1 sigma quantile
band.shade(color='k', alpha=0.3)
# add wider quantile (0.01 .. 0.99)
band.shade(q=0.49, color='gray', alpha=0.2);